In [27]:
import pandas as pd

In [28]:
df = pd.read_csv("/Users/arneshbanerjee/Coding/Code/ml/job_recomendation/jobs_dataset_with_features.csv")

In [29]:
df.shape

(1615940, 2)

In [30]:
df.head()

,Role,Features
0,Social Media Manager,5 to 15 Years Digital Marketing Specialist M.T...
1,Frontend Web Developer,"2 to 12 Years Web Developer BCA HTML, CSS, Jav..."
2,Quality Control Manager,0 to 12 Years Operations Manager PhD Quality c...
3,Wireless Network Engineer,4 to 11 Years Network Engineer PhD Wireless ne...
4,Conference Manager,1 to 12 Years Event Manager MBA Event planning...


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1615940 entries, 0 to 1615939
Data columns (total 2 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   Role      1615940 non-null  object
 1   Features  1615940 non-null  object
dtypes: object(2)
memory usage: 24.7+ MB


In [32]:
df.columns

Index(['Role', 'Features'], dtype='object')

In [33]:
df['Role'].value_counts()

Role
Interaction Designer            20580
Network Administrator           17470
User Interface Designer         14036
Social Media Manager            13945
User Experience Designer        13935
                                ...  
Inventory Control Specialist     3342
Budget Analyst                   3335
Clinical Nurse Manager           3324
Social Science Researcher        3321
Paid Advertising Specialist      3306
Name: count, Length: 376, dtype: int64

In [34]:
min_count = 6500 #setting minimum to 6.5k, dropping casses less than 6k
role_counts = df['Role'].value_counts()
dropped_classes = role_counts[role_counts < min_count].index
filtered_df = df[~df['Role'].isin(dropped_classes)].reset_index(drop=True)

In [35]:
filtered_df['Role'].value_counts()

Role
Interaction Designer          20580
Network Administrator         17470
User Interface Designer       14036
Social Media Manager          13945
User Experience Designer      13935
                              ...  
Benefits Coordinator           6839
Research Analyst               6830
Administrative Coordinator     6803
IT Support Specialist          6799
UI/UX Designer                 6743
Name: count, Length: 61, dtype: int64

In [36]:
len(filtered_df['Role'].value_counts())

61

In [37]:
df = filtered_df.sample(n=10000) #only 10 taken

In [38]:
df.head(10)

,Role,Features
384987,User Interface Designer,5 to 10 Years UX/UI Designer M.Com UI design p...
184151,User Interface Designer,4 to 9 Years UX/UI Designer MBA UI design prin...
199521,Social Media Analyst,3 to 12 Years Social Media Manager B.Tech Soci...
27117,Procurement Analyst,0 to 15 Years Procurement Manager BA Procureme...
177381,Quality Assurance Analyst,0 to 14 Years Software Tester PhD Quality assu...
277000,User Experience Designer,1 to 12 Years UX/UI Designer BCA User-centered...
283820,Customer Support Specialist,2 to 12 Years Customer Success Manager MBA Cus...
471446,Supply Chain Manager,1 to 14 Years Operations Manager MCA Supply ch...
351086,Supply Chain Coordinator,5 to 8 Years Purchasing Agent M.Tech Supply ch...
314219,UX/UI Designer,0 to 8 Years UI Developer BA User experience (...


In [39]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [40]:
#t-t split
X = df['Features']
y = df['Role']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
#tf id vectorisation
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [42]:
#rf classifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train_tfidf, y_train)

y_pred = rf_classifier.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [43]:
def cleanResume(txt):
    cleanText = re.sub(r'http\S+\s', ' ', txt)  # links
    cleanText = re.sub(r'RT|cc', ' ', cleanText)
    cleanText = re.sub(r'#\S+\s', ' ', cleanText)  # special characters
    cleanText = re.sub(r'@\S+', ' ', cleanText)
    cleanText = re.sub(r'[%s]' % re.escape(r"""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', cleanText)
    cleanText = re.sub(r'[^\x00-\x7f]', ' ', cleanText)
    cleanText = re.sub(r'\s+', ' ', cleanText)
    return cleanText

In [44]:
def job_recommendation(resume_text):
    resume_text= cleanResume(resume_text)
    resume_tfidf = tfidf_vectorizer.transform([resume_text])
    predicted_category = rf_classifier.predict(resume_tfidf)[0]
    return predicted_category

In [46]:
import pickle
pickle.dump(rf_classifier,open('models/rf_classifier_job_recommendation.pkl','wb'))
pickle.dump(tfidf_vectorizer,open('models/tfidf_vectorizer_job_recommendation.pkl','wb'))